In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
#import quandl

import cvxportfolio as cp

获取股票价格数据及无风险利率

In [2]:
from zipline.research import returns, volumes
from zipline.data.treasuries_cn import get_treasury_data

In [3]:
# 注意无风险利率选择一年期，然后除每年交易天数244

tickers = ['300001', '000333', '600771', '600645']
start ='2016-01-01'
end ='2018-05-25'

In [4]:
advs = volumes(tickers, start, end)

[2018-05-28 14:01:11.515929] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-25 01:31:00+00:00
最后收盘时间: 2018-05-25 07:00:00+00:00


In [5]:
advs.tail()

,特锐德(300001),美的集团(000333),广誉远(600771),中源协和(600645)
2018-05-21 00:00:00+00:00,4580000.0,17940000.0,2350000.0,4320000.0
2018-05-22 00:00:00+00:00,3230000.0,35120000.0,3100000.0,12300000.0
2018-05-23 00:00:00+00:00,3090000.0,22100000.0,2910000.0,8880000.0
2018-05-24 00:00:00+00:00,4510000.0,22240000.0,1940000.0,7410000.0
2018-05-25 00:00:00+00:00,2860000.0,26150000.0,4020000.0,7400000.0


In [6]:
rets = returns(tickers, start, end)

[2018-05-28 14:01:13.495613] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-25 01:31:00+00:00
最后收盘时间: 2018-05-25 07:00:00+00:00


In [7]:
rets['cash'] = get_treasury_data(start, end)['1year'] / 244.0
rets.head()

,特锐德(300001),美的集团(000333),广誉远(600771),中源协和(600645),cash
2016-01-04 00:00:00+00:00,-0.099644,-0.094482,0.0,-0.099985,0.000096
2016-01-05 00:00:00+00:00,-0.055950,0.054211,0.0,-0.032473,0.000097
2016-01-06 00:00:00+00:00,0.020717,0.008265,0.0,0.047871,0.000097
2016-01-07 00:00:00+00:00,-0.100167,-0.080731,0.0,-0.099966,0.000097
2016-01-08 00:00:00+00:00,-0.021627,0.026527,0.0,0.065368,0.000097


We compute rolling estimates of the first and second moments of the returns using a window of 250 days. We shift them by one unit (so at every day we present the optimizer with only past data).

计算滚动244天

In [8]:
r_hat = rets.rolling(window=244, min_periods=244).mean().shift(1).dropna()
Sigma_hat = rets.rolling(window=244, min_periods=244, closed='neither').cov().dropna()

r_hat.tail()

,特锐德(300001),美的集团(000333),广誉远(600771),中源协和(600645),cash
2018-05-21 00:00:00+00:00,-0.001113,0.002156,0.001664,0.000444,0.000141
2018-05-22 00:00:00+00:00,-0.000842,0.002035,0.001665,0.000487,0.000141
2018-05-23 00:00:00+00:00,-0.000693,0.001857,0.001810,0.000819,0.000141
2018-05-24 00:00:00+00:00,-0.000614,0.001780,0.001805,0.000991,0.000141
2018-05-25 00:00:00+00:00,-0.000689,0.001733,0.001766,0.000936,0.000141


In [9]:
Sigma_hat.tail()

特锐德(300001)  美的集团(000333)  \
2018-05-25 00:00:00+00:00 特锐德(300001)   3.816810e-04  4.955195e-05   
                          美的集团(000333)  4.955195e-05  4.643027e-04   
                          广誉远(600771)   6.266618e-05  5.308990e-05   
                          中源协和(600645)  1.308376e-04 -1.406242e-05   
                          cash         -4.692746e-09  9.682933e-09   

                                         广誉远(600771)  中源协和(600645)  \
2018-05-25 00:00:00+00:00 特锐德(300001)   6.266618e-05  1.308376e-04   
                          美的集团(000333)  5.308990e-05 -1.406242e-05   
                          广誉远(600771)   3.135805e-04  3.150643e-05   
                          中源协和(600645)  3.150643e-05  7.150218e-04   
                          cash         -1.145340e-08 -5.462981e-09   

                                                cash  
2018-05-25 00:00:00+00:00 特锐德(300001)  -4.692746e-09  
                          美的集团(000333)  9.682933e-09  
                          广誉远(600771)  -1.145340e-08  
                          中源协和(600645) -5.462981e-09  
                          cash          6.828097e-11

单周期优化可以获取在限制条件下的交易列表，据此可以计算交易后的股票权重。

# 交易杠杆限制

In [47]:
risk_model = cp.FullSigma(Sigma_hat)
leverage_limit = cp.LeverageLimit(2)

spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[],
                                constraints=[leverage_limit])

In [48]:
t = r_hat.index[2]
p_0 = pd.Series(index=rets.columns, data=10000) # 期初投资组合
z = spo_policy.get_trades(p_0, t)
z

特锐德(300001)    -10000.000506
美的集团(000333)    64999.994332
广誉远(600771)     -9999.998198
中源协和(600645)   -34999.996423
cash            -9999.999205
dtype: float64

In [17]:
import cvxpy as cvx

In [18]:
portfolio = p_0
value = sum(portfolio)

In [19]:
w = portfolio / value
w

特锐德(300001)     0.2
美的集团(000333)    0.2
广誉远(600771)     0.2
中源协和(600645)    0.2
cash            0.2
dtype: float64

In [20]:
z = cvx.Variable(w.size)  # TODO pass index

In [26]:
wplus = w.values + z

In [30]:
from cvxportfolio.utils.data_management import time_locator, null_checker

In [31]:
alpha_term = cvx.sum_entries(cvx.mul_elemwise(
                time_locator(r_hat, t, as_numpy=True), wplus))

In [32]:
alpha_term.is_concave()

True

In [34]:
constraints_obj=[leverage_limit]

In [35]:
costs, constraints = [], []

In [36]:
constraints += [item for item in (con.weight_expr(t, wplus, z, value)
                                          for con in constraints_obj)]

In [37]:
prob = cvx.Problem(
            cvx.Maximize(alpha_term - sum(costs)),
            [cvx.sum_entries(z) == 0] + constraints)

In [38]:
prob.solve()

0.0040909289574366774

In [44]:
z.value

matrix([[-0.20000001],
        [ 1.29999989],
        [-0.19999996],
        [-0.69999993],
        [-0.19999998]])

In [46]:
sum(z.value.A1)

1.3600232051658168e-15

In [42]:
pd.Series(index=portfolio.index, data=(z.value.A1 * value))

特锐德(300001)    -10000.000506
美的集团(000333)    64999.994332
广誉远(600771)     -9999.998198
中源协和(600645)   -34999.996423
cash            -9999.999205
dtype: float64

# 成交量限制

In [12]:
vol_limit = cp.MaxTrade(advs) # 默认最大比例为0.05
spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[],
                                constraints=[vol_limit])

In [13]:
t = r_hat.index[2]
p_0 = pd.Series(index=rets.columns, data=100000000) # 期初投资组合
z = spo_policy.get_trades(p_0, t)
z

特锐德(300001)    -169497.710290
美的集团(000333)    958499.333695
广誉远(600771)      89485.389580
中源协和(600645)         0.000001
cash           -878487.012986
dtype: float64

In [14]:
advs.loc[t] * 0.05

特锐德(300001)     169500.0
美的集团(000333)    958500.0
广誉远(600771)      89500.0
中源协和(600645)         0.0
Name: 2017-01-05 00:00:00+00:00, dtype: float64

返回的是交易量

# 只允许多头

In [15]:
only_long = cp.LongOnly()
spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[],
                                constraints=[only_long])

In [16]:
t = r_hat.index[2]
p_0 = pd.Series(index=rets.columns, data=10000) # 期初投资组合
z = spo_policy.get_trades(p_0, t)
z

特锐德(300001)     -9999.985985
美的集团(000333)    39999.961776
广誉远(600771)     -9999.996998
中源协和(600645)    -9999.990707
cash            -9999.988087
dtype: float64

# 现金非负

In [17]:
cash_limit = cp.LongCash()
spo_policy = cp.SinglePeriodOpt(return_forecast=r_hat, 
                                costs=[],
                                constraints=[cash_limit])

In [18]:
t = r_hat.index[2]
p_0 = pd.Series(index=rets.columns, data=10000) # 期初投资组合
z = spo_policy.get_trades(p_0, t)
z

ERROR:root:The problem is unbounded. Defaulting to no trades


特锐德(300001)     0.0
美的集团(000333)    0.0
广誉远(600771)     0.0
中源协和(600645)    0.0
cash            0.0
dtype: float64